In [1]:
import ee
import os
import pandas as pd
import geemap

In [2]:
map = geemap.Map()
map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
roi = ee.FeatureCollection("users/yehuigeo/poyanghu")
dem = ee.Image("USGS/SRTMGL1_003")
map.addLayer(dem,{},'dem',False)
map.addLayer(roi,{},'roi')
#Function to cloud mask from the pixel_qa band of Landsat 5,7 SR data.  5号、7号星QA波段去云
def cloudMaskL457(image):
    qa = image.select('pixel_qa')
    # If the cloud bit (5) is set and the cloud confidence (7) is high
    # or the cloud shadow bit is set (3), then it's a bad pixel.
    cloud = qa.bitwiseAnd(1 << 5) \
          .And(qa.bitwiseAnd(1 << 7)) \
          .Or(qa.bitwiseAnd(1 << 3))
    # 删除不出现在所有波段的边缘像元
    # Remove edge pixels that don't occur in all bands
    mask2 = image.mask().reduce(ee.Reducer.min())
    return image.updateMask(cloud.Not()).updateMask(mask2)

# Function to cloud mask from the pixel_qa band of Landsat 8 SR data. # 八号星去云
def maskL8sr(image):
    # Bits 3 and 5 are cloud shadow and cloud, respectively. # 第三和第五位分别是云阴影和云
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 5
    # Get the pixel QA band. # 得到QA波段像元
    qa = image.select('pixel_qa')
    # Both flags should be set to zero, indicating clear conditions. # 这两个标志都应该设置为零，表示明确的条件
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
          .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    # Return the masked image, scaled to reflectance, without the QA bands.# 返回掩膜后影像，缩放反射率归为0-1，没有QA波段
    return image.updateMask(mask).copyProperties(image, ["system:time_start"])
def filterBadObs(image):
    azimuth = ee.Number(image.get('SOLAR_AZIMUTH_ANGLE'))
    #selecet SOLAR_ZENITH_ANGLE
    zenith = ee.Number(image.get('SOLAR_ZENITH_ANGLE'))
    #calculate Terrain shadow
    shadow_original = ee.Terrain.hillShadow(dem,azimuth,zenith,100)
    shadow = shadow_original.focal_min(1,"square","pixels",1)
    shadow_band0  = ee.Image(0).where(shadow.eq(1),1)
    # return masked imgage
    return image.updateMask(shadow_band0)
# function: NDVI,LSWI(陆地表面水分指数),MNDWI
def ND_VI(image,b1,b2,bName):
    VI = image.normalizedDifference([b1,b2]).rename(bName)
    return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
# function: EVI(增强型植被指数)
def funEVI(image,B1,B2,B3):
    VI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'BLUE': image.select(B1).multiply(0.0001),   
      'RED': image.select(B2).multiply(0.0001),   
      'NIR': image.select(B3).multiply(0.0001)
    }).rename('EVI')
    return VI.updateMask(VI.gt(-1).And(VI.lt(1)))
# function: Add VIs to image 将各类指数加入影像中
def addLandsatVIs(img):
    NDVI = ND_VI(img,'B4','B3','NDVI')
    EVI = funEVI(img,'B1','B3','B4')
    mNDWI = ND_VI(img,'B2','B5','mNDWI')
    return img.addBands(NDVI).addBands(EVI).addBands(mNDWI)
    # water detection 水提取 MNDWI > NDVI或MNDWI > EVI 和 EVI < 0.1
def Water(img):
    return img.select('mNDWI').gt(img.select('EVI'))\
    .Or(img.select('mNDWI').gt(img.select('NDVI')))\
    .And(img.select('EVI').lt(0.1))
# Non-Water detection  非水提取
def NonWater(image):
    return image.select('mNDWI').lte(image.select('EVI'))\
    .And(image.select('mNDWI').lte(image.select('NDVI')))\
    .Or(image.select('EVI').gte(0.1))

In [ ]:
# 起始、终止时间
# 丰水期
for year in range(2018,2021):
    START_DATE  = str(year) + '-06'
    END_DATE = str(year) + '-10'
    #get Landsat5 collction B1:blue,B2:green,B3:red,B4:NIR
    collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
        .filterBounds(roi) \
        .map(filterBadObs)\
        .filterDate(START_DATE,END_DATE ) \
        .map(cloudMaskL457) \
        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
        .map(addLandsatVIs)
    #get Landsat7 collction B1:blue,B2:green,B3:red,B4:NIR
    collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
        .filterBounds(roi) \
        .map(filterBadObs)\
        .filterDate(START_DATE, END_DATE ) \
        .map(cloudMaskL457) \
        .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
        .map(addLandsatVIs)
    #get Landsat8 collction
    collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
        .filterBounds(roi) \
        .map(filterBadObs)\
        .filterDate(START_DATE,END_DATE ) \
        .map(maskL8sr) \
        .select( 
          ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
          ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])\
        .map(addLandsatVIs)
        # Merge to a single collection 融合成一个数据集
    col1 = ee.ImageCollection(collection5.merge(collection7))
    collection = ee.ImageCollection(col1.merge(collection8))
    # 得到有水像元和无水像元两种的数量
    # water frequency and quality band (number of good observations)
    total_Pos = collection.map(Water).sum()
    total_Neg = collection.map(NonWater).sum()
    # water frequency  淹没频率 有水的像元除以所有的像元，得到一个比例
    flooding_freq = (total_Pos.divide(total_Pos.add(total_Neg))).rename("flooding_freq")
    # 根据淹没频率将水体分类，gt >,gte >=,It <,Ite <=
    # 长 == 永久性水体
    long_water = ee.Image(1).updateMask(flooding_freq.gte(0.75)).clip(roi.geometry()).rename("long_water")

#     # muti_short: water frequency >= 0.25 and water frequency < 0.75   == 季节性
    short_water = ee.Image(1).updateMask(flooding_freq.gte(0.25).And(flooding_freq.lt(0.75))).clip(roi.geometry()).rename("short_water")

    # muti_max:water frequency >= 0.25 and water frequency <= 1 
    max_water =  ee.Image(1).updateMask(flooding_freq.gte(0.25).And(flooding_freq.lte(1))).clip(roi.geometry()).rename("max_water")
    
#     geemap.ee_export_image_to_drive(
#     max_water,
#     description = "f_max" + "_" + str(year),
#     region=roi.geometry(),
#     scale=30,
#     crs='EPSG:4326',
#     folder='Water'
#     )
#     geemap.ee_export_image_to_drive(
#     long_water,
#     description = "f_long" + "_" + str(year),
#     region=roi.geometry(),
#     scale=30,
#     crs='EPSG:4326',
#     folder='Water'
#     )
#     geemap.ee_export_image_to_drive(
#     short_water,
#     description = "f_short" + "_" + str(year),
#     region=roi.geometry(),
#     scale=30,
#     crs='EPSG:4326',
#     folder='Water'
#     )

    # Ephemeral surface water: water frequency < 0.25
#     eph_water = ee.Image(1).updateMask(flooding_freq.lt(0.25)).clip(roi.geometry()).rename("eph_water")
#     long_water_area = long_water.multiply(900).reduceRegion(**{
#                         'reducer': ee.Reducer.sum(),
#                         'geometry': roi.geometry(),
#                         'scale': 30,
#                         'maxPixels': 1e13,
#                         'tileScale':8})
#     df1 = pd.DataFrame(long_water_area.getInfo(),index = [0])
#     print(df1)
#     out_path1 = r"C:\Users\小骆\Desktop\鄱阳湖动态分析\更改月份后面积\丰-long"
#     out1 = out_path1 + os.sep + str(year) + '_' + 'flong_water' + '.txt'
#     df1.to_csv(out1,sep=',',index=False,header=True)
   
#     short_water_area = short_water.multiply(900).reduceRegion(**{
#                         'reducer': ee.Reducer.sum(),
#                         'geometry': roi.geometry(),
#                         'scale': 30,
#                         'maxPixels': 1e13,
#                         'tileScale':8})
#     df2 = pd.DataFrame(short_water_area.getInfo(),index = [0])
#     print(df2)
#     out_path2 = r"C:\Users\小骆\Desktop\鄱阳湖动态分析\更改月份后面积\丰-sea"
#     out2 = out_path2 + os.sep + str(year) + '_' + 'fshort_water' + '.txt'
#     df2.to_csv(out2,sep=',',index=False,header=True)

#     max_water_area = max_water.multiply(900).reduceRegion(**{
#                         'reducer': ee.Reducer.sum(),
#                         'geometry': roi.geometry(),
#                         'scale': 30,
#                         'maxPixels': 1e13,
#                         'tileScale': 8
#                        })
#     df3 = pd.DataFrame(max_water_area.getInfo(),index = [0])
#     print(df3)
#     out_path3 = r"C:\Users\小骆\Desktop\丰"
#     out3 = out_path3 + os.sep + str(year)  +  '_' + 'fmax_water' +  '.txt'
#     df3.to_csv(out3,sep=',',index=False,header=True)

In [5]:
# for year in range(1986,2021):
#     START_DATE  = str(year) + '-06'
#     END_DATE = str(year) + '-10'
#     #get Landsat5 collction B1:blue,B2:green,B3:red,B4:NIR
#     collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
#         .filterBounds(roi) \
#         .map(filterBadObs)\
#         .filterDate(START_DATE,END_DATE ) \
#         .map(cloudMaskL457) \
#         .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
#         .map(addLandsatVIs)
#      #get Landsat7 collction B1:blue,B2:green,B3:red,B4:NIR
#     collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
#         .filterBounds(roi) \
#         .map(filterBadObs)\
#         .filterDate(START_DATE, END_DATE ) \
#         .map(cloudMaskL457) \
#         .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
#         .map(addLandsatVIs)
#     #get Landsat8 collction
#     collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#         .filterBounds(roi) \
#         .map(filterBadObs)\
#         .filterDate(START_DATE,END_DATE ) \
#         .map(maskL8sr) \
#         .select( 
#           ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
#           ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])\
#         .map(addLandsatVIs)
#         # Merge to a single collection 融合成一个数据集
#     col1 = ee.ImageCollection(collection5.merge(collection7))
#     collection = ee.ImageCollection(col1.merge(collection8))

#     names = collection.aggregate_array('system:index').getInfo()
#     print(names)
#     print(collection.size().getInfo())

['1_1_LT05_121039_19860605', '1_1_LT05_121039_19860723', '1_1_LT05_121039_19860808', '1_1_LT05_121040_19860621', '1_1_LT05_121040_19860723', '1_1_LT05_121040_19860808', '1_1_LT05_122039_19860730']
7
['1_1_LT05_121039_19870811', '1_1_LT05_121039_19870827', '1_1_LT05_121040_19870624', '1_1_LT05_121040_19870811', '1_1_LT05_121040_19870827', '1_1_LT05_121040_19870912', '1_1_LT05_121040_19870928', '1_1_LT05_122039_19870615', '1_1_LT05_122039_19870818', '1_1_LT05_122039_19870919', '1_1_LT05_122040_19870818', '1_1_LT05_122040_19870919']
12
['1_1_LT05_121039_19880626', '1_1_LT05_121039_19880813', '1_1_LT05_121040_19880610', '1_1_LT05_121040_19880626', '1_1_LT05_121040_19880728', '1_1_LT05_121040_19880813', '1_1_LT05_122039_19880601', '1_1_LT05_122039_19880703', '1_1_LT05_122039_19880804', '1_1_LT05_122039_19880921', '1_1_LT05_122040_19880601', '1_1_LT05_122040_19880703', '1_1_LT05_122040_19880804', '1_1_LT05_122040_19880921']
14
['1_1_LT05_121039_19890613', '1_1_LT05_121039_19890715', '1_1_LT0

In [ ]:
max_water.getInfo()

In [6]:
# for year in range(1986,2021):
#     if year + 1 in range(1986,2022):
#         START_DATE  = str(year) + '-11'
#         END_DATE = str(year + 1) + '-03'

#     #get Landsat5 collction B1:blue,B2:green,B3:red,B4:NIR
#     collection5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
#         .filterBounds(roi) \
#         .map(filterBadObs)\
#         .filterDate(START_DATE,END_DATE ) \
#         .map(cloudMaskL457) \
#         .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
#         .map(addLandsatVIs)
#      #get Landsat7 collction B1:blue,B2:green,B3:red,B4:NIR
#     collection7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
#         .filterBounds(roi) \
#         .map(filterBadObs)\
#         .filterDate(START_DATE, END_DATE ) \
#         .map(cloudMaskL457) \
#         .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7','pixel_qa'])\
#         .map(addLandsatVIs)
#     #get Landsat8 collction
#     collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
#         .filterBounds(roi) \
#         .map(filterBadObs)\
#         .filterDate(START_DATE,END_DATE ) \
#         .map(maskL8sr) \
#         .select( 
#           ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa']
#           ,['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa'])\
#         .map(addLandsatVIs)
#         # Merge to a single collection 融合成一个数据集
#     col1 = ee.ImageCollection(collection5.merge(collection7))
#     collection = ee.ImageCollection(col1.merge(collection8))

#     names = collection.aggregate_array('system:index').getInfo()
#     print(names)
#     print(collection.size().getInfo())

['1_1_LT05_121039_19870131', '1_1_LT05_121040_19861128', '1_1_LT05_121040_19870131', '1_1_LT05_122039_19861103', '1_1_LT05_122039_19861221', '1_1_LT05_122039_19870106', '1_1_LT05_122039_19870207', '1_1_LT05_122040_19861103', '1_1_LT05_122040_19861221', '1_1_LT05_122040_19870106', '1_1_LT05_122040_19870207']
11
['1_1_LT05_121039_19871201', '1_1_LT05_121039_19880118', '1_1_LT05_121040_19871201', '1_1_LT05_121040_19871217', '1_1_LT05_122039_19871208', '1_1_LT05_122039_19880125', '1_1_LT05_122040_19871208']
7
['1_1_LT05_121039_19881101', '1_1_LT05_121039_19881203', '1_1_LT05_121039_19890104', '1_1_LT05_121040_19881101', '1_1_LT05_121040_19881203', '1_1_LT05_121040_19890104', '1_1_LT05_122039_19881108', '1_1_LT05_122039_19881124', '1_1_LT05_122039_19890127', '1_1_LT05_122040_19881108', '1_1_LT05_122040_19881124', '1_1_LT05_122040_19890127', '1_1_LT05_122040_19890228']
13
['1_1_LT05_121039_19891120', '1_1_LT05_121039_19900123', '1_1_LT05_121040_19891104', '1_1_LT05_121040_19891120', '1_1_LT0